# Noise-free optimization with Expected Improvement

In [ ]:
import numpy as np
import tensorflow as tf
import trieste

import gpflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow.utilities import print_summary, positive, set_trainable
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter


np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
tf.config.run_functions_eagerly(True)
# define database '297' or '388'
N = 297

# define numbe rof training dataset
n_train = 200

n_iter = 1

n_max = 1
# Read the data
if N == 297:
    data_E = '../../../data/297_energy.txt'
    data_pd = '../../../data/297_octonion_pd.txt'
    data_axes = '../../../data/sigma3_data.txt'
    if n_train > N:
        print('The numbe rof trainign datset should be smaller than 297')
else:
    data_E = '../../../data/energy_olms.txt'
    data_pd = '../../../data/pd_olms.txt'
    if n_train > N:
        print('The numbe rof trainign datset should be smaller than 388')
def _octonion_dist(X, X2):
    X = tf.reshape(X, [-1,1])
    pd = np.loadtxt(data_pd)
    X2 = tf.reshape(X2, [-1,1])
    dist0 = np.zeros((len(X), len(X2)))
    dist = tf.Variable(dist0) # Use variable 
    for i in range(len(X)):
        init_val = int(X[i].numpy())
        for j in range(len(X2)):
            fin_val = int(X2[j].numpy())
            dist0[i,j] = pd[init_val, fin_val]
    dist.assign(dist0)
    return dist

def _octonion_dist_single(X):
    X = tf.reshape(X, [-1,1])
    pd = np.loadtxt(data_pd)
    dist0 = np.zeros((len(X)))
    dist = tf.Variable(dist0) # Use variable 
    for i in range(len(X)):
        init_val = int(X[i].numpy())

        dist0[i] = pd[init_val, init_val]
    dist.assign(dist0)
    return dist

class GBKernel(gpflow.kernels.Kernel):
    def __init__(self, variance=1, lengthscales=.2, **kwargs):
        super().__init__(**kwargs)
        self.variance = gpflow.Parameter(variance, transform=positive())
        self.lengthscales = gpflow.Parameter(lengthscales, transform=positive())
        self._validate_ard_active_dims(self.lengthscales)
    
    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        a = self.variance * tf.exp(-_octonion_dist(X, X2)/self.lengthscales)
        tf.debugging.check_numerics(self.lengthscales, 'hi')
        return a

    def K_diag(self, X):
        a = self.variance * tf.exp(-_octonion_dist_single(X)/self.lengthscales)
        tf.debugging.check_numerics(self.lengthscales, 'hi')
        return a

In [ ]:
#input data
N = 297
id_gb = np.linspace(0,N-1,N)
id_gb = tf.convert_to_tensor(id_gb, dtype=tf.float64)

## Describe the problem
In this example, we look to find the minimum value of the two-dimensional Branin function over the hypercube $[0, 1]^2$. We can represent the search space using a `Box`, and plot contours of the Branin over this space.

In [ ]:

tf.config.run_functions_eagerly(True)
def gb_func(id_gb):
    """
    x is the id of gbs
    output is the energy
    """
    y0 = -np.loadtxt('../../../data/E_cleav.txt')
#     y0 = -np.loadtxt('../../../data/energy_olms.txt')
    mean_y = np.mean(y0)
    std_y = np.sqrt(np.var(y0))
    y = (y0 - mean_y) / std_y
    id_gb = id_gb.numpy()
    Y = y[id_gb.astype(int)]
    return Y



In [ ]:

input_data =tf.convert_to_tensor(id_gb)
inp = tf.reshape(input_data, [-1,1])
search_space = trieste.space.DiscreteSearchSpace(inp)

## Sample the observer over the search space

Sometimes we don't have direct access to the objective function. We only have an observer that indirectly observes it. In _Trieste_, the observer outputs a number of datasets, each of which must be labelled so the optimization process knows which is which. In our case, we only have one dataset, the objective. We'll use _Trieste_'s default label for single-model setups, `OBJECTIVE`. We can convert a function with `branin`'s signature to a single-output observer using `mk_observer`.

The optimization procedure will benefit from having some starting data from the objective function to base its search on. We sample five points from the search space and evaluate them on the observer.

In [ ]:
from trieste.acquisition.rule import OBJECTIVE

observer = trieste.utils.objectives.mk_observer(gb_func, OBJECTIVE)

num_initial_points = 2
initial_query_points = np.array([18, 209])
# initial_query_points = search_space.sample(num_initial_points)
initial_query_points = tf.convert_to_tensor(initial_query_points, dtype=tf.float64)
initial_query_points = tf.reshape(initial_query_points, [-1,1])

initial_data = observer(initial_query_points)

In [ ]:
initial_query_points.shape
# initial_query_points0

## Model the objective function

The Bayesian optimization procedure estimates the next best points to query by using a probabilistic model of the objective. We'll use Gaussian process regression for this, provided by GPflow. The model will need to be trained on each step as more points are evaluated, so we'll package it with GPflow's Scipy optimizer.

Just like the data output by the observer, the optimization process assumes multiple models, so we'll need to label the model in the same way.

In [ ]:
import gpflow

def build_model(data):
    variance = tf.math.reduce_variance(data.observations)
    kernel = GBKernel()
    gpr = gpflow.models.GPR(data.astuple(), kernel, noise_variance=1e-5)
#     print_summary(gpr)
    gpflow.set_trainable(gpr.kernel.lengthscales, True)
    gpflow.set_trainable(gpr.kernel.variance, True)
    gpflow.set_trainable(gpr.likelihood, False)

    return {OBJECTIVE: {
        "model": gpr,
        "optimizer": gpflow.optimizers.Scipy(),
        "optimizer_args": {
            "minimize_args": {"options": dict(maxiter=100)},
        },
    }}

model = build_model(initial_data[OBJECTIVE])

## Run the optimization loop

We can now run the Bayesian optimization loop by defining a `BayesianOptimizer` and calling its `optimize` method.

The optimizer uses an acquisition rule to choose where in the search space to try on each optimization step. We'll use the default acquisition rule, which is Efficient Global Optimization with Expected Improvement.

We'll run the optimizer for fifteen steps.

The optimization loop catches errors so as not to lose progress, which means the optimization loop might not complete and the data from the last step may not exist. Here we'll handle this crudely by asking for the data regardless, using `.try_get_final_datasets()`, which will re-raise the error if one did occur. For a review of how to handle errors systematically, there is a [dedicated tutorial](recovering_from_errors.ipynb). Finally, like the observer, the optimizer outputs labelled datasets, so we'll get the (only) dataset here by indexing with tag `OBJECTIVE`.

In [ ]:
bo = trieste.bayesian_optimizer.BayesianOptimizer(observer, search_space)
# print_summary(model)
result = bo.optimize(50, initial_data, model)
dataset = result.try_get_final_datasets()[OBJECTIVE]

## Explore the results

We can now get the best point found by the optimizer. Note this isn't necessarily the point that was last evaluated.

In [ ]:
query_points = dataset.query_points.numpy()
observations = dataset.observations.numpy()
y0 = -np.loadtxt('../../../data/E_cleav.txt')
mean_y = np.mean(y0)
std_y = np.sqrt(np.var(y0))

predicted = (observations * std_y + mean_y)
arg_min_idx = tf.squeeze(tf.argmin(predicted, axis=0))

print(f"grain boundary id: {int(query_points[arg_min_idx, :]) + 1}")
print(f"Predicted value: {-predicted[arg_min_idx, :]}")
print(f"Optimization step: {arg_min_idx}")



In [ ]:
import matplotlib.pyplot as plt
from util.plotting import plot_regret

fig, ax = plt.subplots(figsize=(10, 5))
plot_regret(predicted, ax, num_init=num_initial_points, idx_best=arg_min_idx)



In [ ]:
# id_gb = np.linspace(0,N-1,N)
# plt.scatter(id_gb, y0)

In [ ]:

ls_list = [
    step.models[OBJECTIVE].model.kernel.lengthscales.numpy()  # type: ignore
    for step in result.history + [result.final_result.unwrap()]
]

var_list = [
    step.models[OBJECTIVE].model.kernel.variance.numpy()  # type: ignore
    for step in result.history + [result.final_result.unwrap()]
]
ls = np.array(ls_list)
var = np.array(var_list)

plt.figure(figsize=(10,5))
ax1 = plt.subplot(1,2,1)
ax2 = plt.subplot(1,2,2)
fig.suptitle('Horizontally stacked subplots')
ax1.plot(ls)
ax1.set_xlabel('step')
ax1.set_ylabel('lengthscale')
ax2.plot(var)
ax2.set_xlabel('step')
ax2.set_ylabel('variance')
plt.subplots_adjust(wspace=0.4)

gpflow.utilities.print_summary(result.try_get_final_models()[OBJECTIVE].model)